In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import datetime
import gc
import os
import pickle

## Constantes

Se analizan los datos de exportaciones promediando una VENTANA de años que finalizan en el año ANIO. Es decir, si se deasea realizar el promedio de las exportaciones entre 2014 y 2018 inclusive, VENTANA debe ser 5 y ANIO debe ser 2018.

In [2]:
ANIO = 2018 # último año de la VENTANA de años de datos para promediar y exportar
VENTANA = 5 # (ventana de promedio de datos, en años, se cuenta desde ANIO inclusive para atrás)
DATASETS_DIR = './datasets/' # si guardas los datos en el mismo directorio que esta notebook, cambiar por './'

In [3]:
df = pd.read_csv(DATASETS_DIR + "country_hsproduct6digit_year.tab", sep='\t',
                 dtype={'hs_product_code': str},
                 usecols=[2, 3, 7, 8])#, low_memory=False)

In [4]:
lista_paises = pd.read_csv(DATASETS_DIR + "lista_paises_iso3.csv", low_memory=False)

lista_paises = lista_paises['ISO3'].tolist()
#lista_paises

## Exploración

In [5]:
df.head()

,year,export_value,location_code,hs_product_code
0,1995,18008.0,ABW,010111
1,1996,0.0,ABW,010111
2,1997,NaN,ABW,010111
3,1998,NaN,ABW,010111
4,1999,14510.0,ABW,010111


In [6]:
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30385560 entries, 0 to 30385559
Data columns (total 4 columns):
 #   Column           Dtype  
---  ------           -----  
 0   year             int64  
 1   export_value     float64
 2   location_code    object 
 3   hs_product_code  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 927.3+ MB


year                      0
export_value       12332791
location_code             0
hs_product_code           0
dtype: int64

In [7]:
df.year.value_counts()

2018    1266065
2017    1266065
1996    1266065
1997    1266065
1998    1266065
1999    1266065
2000    1266065
2001    1266065
2002    1266065
2003    1266065
2004    1266065
2005    1266065
2006    1266065
2007    1266065
2008    1266065
2009    1266065
2010    1266065
2011    1266065
2012    1266065
2013    1266065
2014    1266065
2015    1266065
2016    1266065
1995    1266065
Name: year, dtype: int64

## Filtrado

In [7]:
# Filtra por listado de países

df.drop(df.loc[~df['location_code'].isin(lista_paises)].index, inplace = True)
df.location_code.nunique()

122

In [8]:
#Filtra por productos extraños

hs_malos = ['XXXXXX', 'financial', 'ict', 'transport', 'travel', 'unspecified', '9999AA']
df.drop(df.loc[df['hs_product_code'].isin(hs_malos)].index, inplace = True)
df.dtypes

year                 int64
export_value       float64
location_code       object
hs_product_code     object
dtype: object

## Cálculo de ventanas para suavizar exportaciones

In [9]:
#agregamos una columna que vamos a usar como indice para mergear, juntando location_code con hs_product_code
df['id'] = df['location_code'] + '-' + df['hs_product_code'].astype(str)
df.export_value.fillna(0, inplace = True)
df

,year,export_value,location_code,hs_product_code,id
48,1995,0.0,AGO,010111,AGO-010111
49,1996,0.0,AGO,010111,AGO-010111
50,1997,0.0,AGO,010111,AGO-010111
51,1998,0.0,AGO,010111,AGO-010111
52,1999,0.0,AGO,010111,AGO-010111
...,...,...,...,...,...
30348859,2014,14687625.0,ZWE,999999,ZWE-999999
30348860,2015,6388037.0,ZWE,999999,ZWE-999999
30348861,2016,2828480.0,ZWE,999999,ZWE-999999
30348862,2017,76244103.0,ZWE,999999,ZWE-999999


In [10]:
#Vamos filtrando por año para formar una matriz general con los años en columnas, y así sacar más facil los promedios

a = ANIO

df_acum = df.loc[df.year == a].copy()

for x in range(1, VENTANA):
    a = a - 1
    df_acum = df_acum.merge(df.loc[df.year == a].rename(columns={"export_value": a}), how='left', on='id')
    df_acum['export_value'] = df_acum['export_value'] + df_acum[a]
    df_acum.drop(columns=[a], inplace=True)

df_acum.drop(columns=['id','year','location_code_x', 'hs_product_code_x', 
                      'year_x','location_code_y','hs_product_code_y','year_y'],
             inplace = True)    
df_acum['export_value'] = df_acum['export_value']/VENTANA

df_acum

,export_value,location_code,hs_product_code
0,0.000000e+00,AGO,010111
1,0.000000e+00,ALB,010111
2,4.352780e+07,ARE,010111
3,7.145202e+06,ARG,010111
4,1.067275e+08,AUS,010111
...,...,...,...
614631,7.702067e+09,VNM,999999
614632,4.888316e+06,YEM,999999
614633,3.845820e+09,ZAF,999999
614634,2.267060e+07,ZMB,999999


In [11]:
#Filtramos por productos que no registran exportaciones

#Agrupo por producto y saco aquellos que no tienen exportaciones
productos_agrup = df_acum.groupby(['hs_product_code']).sum()
productos_agrup.drop(productos_agrup.loc[productos_agrup['export_value'] == 0].index, inplace = True)
productos = productos_agrup.index.tolist()

df_acum.drop(df_acum.loc[~df_acum['hs_product_code'].isin(productos)].index, inplace=True)
df_acum

,export_value,location_code,hs_product_code
0,0.000000e+00,AGO,010111
1,0.000000e+00,ALB,010111
2,4.352780e+07,ARE,010111
3,7.145202e+06,ARG,010111
4,1.067275e+08,AUS,010111
...,...,...,...
614631,7.702067e+09,VNM,999999
614632,4.888316e+06,YEM,999999
614633,3.845820e+09,ZAF,999999
614634,2.267060e+07,ZMB,999999


## Cálculo RCA

In [12]:
#Matriz de monto exportado por pais y producto. Sería Xcp.

df_base = df_acum.copy()
del df_acum
gc.collect()

df_base

,export_value,location_code,hs_product_code
0,0.000000e+00,AGO,010111
1,0.000000e+00,ALB,010111
2,4.352780e+07,ARE,010111
3,7.145202e+06,ARG,010111
4,1.067275e+08,AUS,010111
...,...,...,...
614631,7.702067e+09,VNM,999999
614632,4.888316e+06,YEM,999999
614633,3.845820e+09,ZAF,999999
614634,2.267060e+07,ZMB,999999


In [13]:
#Vector de valores exportados totales por país (sum p Xcp)

df_agrup_c = df_base.groupby(['location_code']).sum()
df_agrup_c

,export_value
location_code,
AGO,3.745867e+10
ALB,2.394034e+09
ARE,2.550475e+11
ARG,5.993725e+10
AUS,2.182895e+11
...,...
VNM,2.026418e+11
YEM,2.584359e+09
ZAF,9.924177e+10


In [14]:
#Calculamos el ratio que representa la exportación de un determinado producto en la canasta de exportaciones de su país

#PASOS
#left join de df_base con df_agrup_c por location_code 
#dividir export_value por producto y país, por el export_value del país en su conjunto

df_pais = df_base.merge(df_agrup_c, how='left', on='location_code')
df_pais['porc_pais']=df_pais['export_value_x']/df_pais['export_value_y']
#df_pais = df_pais.drop(columns=['export_value_x', 'export_value_y'])
df_pais

,export_value_x,location_code,hs_product_code,export_value_y,porc_pais
0,0.000000e+00,AGO,010111,3.745867e+10,0.000000
1,0.000000e+00,ALB,010111,2.394034e+09,0.000000
2,4.352780e+07,ARE,010111,2.550475e+11,0.000171
3,7.145202e+06,ARG,010111,5.993725e+10,0.000119
4,1.067275e+08,AUS,010111,2.182895e+11,0.000489
...,...,...,...,...,...
593403,7.702067e+09,VNM,999999,2.026418e+11,0.038008
593404,4.888316e+06,YEM,999999,2.584359e+09,0.001892
593405,3.845820e+09,ZAF,999999,9.924177e+10,0.038752
593406,2.267060e+07,ZMB,999999,6.472629e+09,0.003503


In [15]:
#vuela
#Probamos que los ratios sumen 1 para cada país

df_pais_sum = df_pais.groupby(['location_code']).sum()
print('Porcentaje total de exportaciones por país es igual a uno?: ', 
      np.isclose(df_pais_sum['porc_pais'], 1.0).all())

Porcentaje total de exportaciones por país es igual a uno?:  True


In [16]:
#Vector de valores exportados totales por producto (sum c Xcp)

df_agrup_p = df_base.groupby(['hs_product_code']).sum()
df_agrup_p

,export_value
hs_product_code,
010111,1.482799e+09
010119,1.000056e+09
010120,2.123955e+07
010210,1.140194e+09
010290,6.904904e+09
...,...
970300,4.082718e+09
970400,1.334660e+08
970500,1.221319e+09


In [17]:
#Monto total exportado para todos los paises y productos

df_agrup_cp = df_agrup_p['export_value'].sum()
df_agrup_cp

16073059785703.6

In [18]:
#Cálculo del ratio que representan las exportaciones de un producto sobre el total de exportaciones (para todos los productos y países)
df_agrup_p['export_total'] = df_agrup_cp
df_agrup_p['porc_prod'] = df_agrup_p['export_value']/df_agrup_p['export_total']
df_agrup_p

,export_value,export_total,porc_prod
hs_product_code,,,
010111,1.482799e+09,1.607306e+13,0.000092
010119,1.000056e+09,1.607306e+13,0.000062
010120,2.123955e+07,1.607306e+13,0.000001
010210,1.140194e+09,1.607306e+13,0.000071
010290,6.904904e+09,1.607306e+13,0.000430
...,...,...,...
970300,4.082718e+09,1.607306e+13,0.000254
970400,1.334660e+08,1.607306e+13,0.000008
970500,1.221319e+09,1.607306e+13,0.000076


In [19]:
#Unimos ambas tablas para tener ambos ratios unificados. Calculamos RCA como la división entre los dos,
# y M cómo un booleano que vale 1 en caso de que RCA≥1 y 0 en el resto de los casos.

#left join de df_pais con df_agrup_p por hs_product_code, agregando campo porc_export
#calculamos RCA y M

df_RCA = df_pais.merge(df_agrup_p, how='left', on='hs_product_code')
df_RCA['RCA'] = df_RCA['porc_pais'] / df_RCA['porc_prod']

df_RCA['M'] = np.where(df_RCA['RCA'] >= 1, 1, 0)

df_RCA.drop(columns=['porc_pais', 'export_value', 'export_total', 
                     'porc_prod','export_value_y'],
            inplace=True)
df_RCA.rename(columns={"export_value_x": "export_value"}, inplace=True)
    
df_RCA

,export_value,location_code,hs_product_code,RCA,M
0,0.000000e+00,AGO,010111,0.000000,0
1,0.000000e+00,ALB,010111,0.000000,0
2,4.352780e+07,ARE,010111,1.849958,1
3,7.145202e+06,ARG,010111,1.292213,1
4,1.067275e+08,AUS,010111,5.299802,1
...,...,...,...,...,...
593403,7.702067e+09,VNM,999999,1.064169,1
593404,4.888316e+06,YEM,999999,0.052959,0
593405,3.845820e+09,ZAF,999999,1.084993,1
593406,2.267060e+07,ZMB,999999,0.098065,0


In [20]:
df_RCA.dtypes

export_value       float64
location_code       object
hs_product_code     object
RCA                float64
M                    int64
dtype: object

In [21]:
# crea columnas de indices fijos para paises y productos para poder transportar la matriz a numpy:
locations = df_RCA.location_code.unique()
products = df_RCA.hs_product_code.unique()

df_RCA['loc_int'] = df_RCA.location_code.apply(lambda x: np.where(locations == x)[0][0])
df_RCA['prod_int'] = df_RCA.hs_product_code.apply(lambda x: np.where(products == x)[0][0])
df_RCA

,export_value,location_code,hs_product_code,RCA,M,loc_int,prod_int
0,0.000000e+00,AGO,010111,0.000000,0,0,0
1,0.000000e+00,ALB,010111,0.000000,0,1,0
2,4.352780e+07,ARE,010111,1.849958,1,2,0
3,7.145202e+06,ARG,010111,1.292213,1,3,0
4,1.067275e+08,AUS,010111,5.299802,1,4,0
...,...,...,...,...,...,...,...
593403,7.702067e+09,VNM,999999,1.064169,1,117,4863
593404,4.888316e+06,YEM,999999,0.052959,0,118,4863
593405,3.845820e+09,ZAF,999999,1.084993,1,119,4863
593406,2.267060e+07,ZMB,999999,0.098065,0,120,4863


In [22]:
# crea matriz Mcp en numpy a partir de dataframe df_RCA

# matriz de ceros del tamanio correcto
Mcp = np.zeros((df_RCA.location_code.nunique(), df_RCA.hs_product_code.nunique()),
               dtype=np.int64)

# llena de unos en los lugares correspondientes segun df_RCA
for row in df_RCA.loc[df_RCA.M == 1].iterrows():
    Mcp[row[1].loc_int, row[1].prod_int] = 1

In [23]:
print('Cuantos elementos tienen RCA > 1 segun df_RCA:', len(df_RCA.loc[df_RCA.M == 1]))
print('Cuantos elementos tienen RCA > 1 segun Mcp:   ', Mcp.sum())

Cuantos elementos tienen RCA > 1 segun df_RCA: 76902
Cuantos elementos tienen RCA > 1 segun Mcp:    76902


In [24]:
# crea matriz de exportaciones en valores netos

export_values = np.zeros((df_RCA.location_code.nunique(), df_RCA.hs_product_code.nunique()),
               dtype=np.float64)

# llena de export_value en los lugares correspondientes segun df_RCA
for row in df_RCA.loc[df_RCA.export_value != 0.0].iterrows():
    export_values[row[1].loc_int, row[1].prod_int] = row[1].export_value

In [25]:
print('Cuanta exportacion total hay segun df_RCA:', df_RCA.export_value.sum())
print('Cuanta exportacion total hay segun Mcp:   ', export_values.sum())

Cuanta exportacion total hay segun df_RCA: 16073059785703.602
Cuanta exportacion total hay segun Mcp:    16073059785703.598


In [26]:
# crea matriz de RCAs

RCA = np.zeros((df_RCA.location_code.nunique(), df_RCA.hs_product_code.nunique()),
               dtype=np.float64)

# llena de RCA en los lugares correspondientes segun df_RCA
for row in df_RCA.loc[df_RCA.RCA != 0.0].iterrows():
    RCA[row[1].loc_int, row[1].prod_int] = row[1].RCA

In [27]:
print('Cuanto RCA total hay segun df_RCA:', df_RCA.RCA.sum())
print('Cuanto RCA total hay segun Mcp:   ', RCA.sum())

Cuanto RCA total hay segun df_RCA: 682397.6684484596
Cuanto RCA total hay segun Mcp:    682397.6684484596


## Guarda serializables de datos útiles en carpeta 'serializables'

In [28]:
if os.path.isdir('./serializables'):
    print('el directorio "serializables" ya existe, ojo con sobreescribir archivos que ya existen')
else:
    !mkdir serializables
    print('directorio "serializables" no existia, creado ahora')

directorio "serializables" no existia, creado ahora


In [29]:
with open(f'serializables/Mcp_{ANIO}_{VENTANA}.pkl','wb') as f:
    pickle.dump(Mcp, f)

In [30]:
with open(f'serializables/export_values_{ANIO}_{VENTANA}.pkl','wb') as f:
    pickle.dump(export_values, f)

In [31]:
with open(f'serializables/RCA_{ANIO}_{VENTANA}.pkl','wb') as f:
    pickle.dump(RCA, f)

In [32]:
with open(f'serializables/locations_{ANIO}_{VENTANA}.pkl','wb') as f:
    pickle.dump(locations, f)

In [33]:
with open(f'serializables/products_{ANIO}_{VENTANA}.pkl','wb') as f:
    pickle.dump(products, f)